# HHL Algorithm

The research I’m proposing is based on a paper I found when I was searching through the Quantum Zoo site [1]. Originally, I had planned to do a paper on a quantum machine learning algorithm, and I found a paper that was describing how there is more to gaining a speedup in machine learning than just throwing the problem at a quantum computer. The paper was called Quantum Machine Learning Algorithms: Read the Fine Print which explained that most quantum algorithms being used for machine learning are in most part either derived from or wholly using the HHL algorithm which was developed by the researcher’s colleagues [2]. I decided that what was better than learning from a derivative was researching the more substantive and foundational source algorithm. The HHL algorithm is named after the authors of the Quantum algorithm for linear systems of equations paper, and is described as an algorithm that is more efficient than classical algorithms at solving a linear systems of equations [3]. For a given linear system  $$Ax=b$$
⃗

They consider the case where one doesn’t need to know the solution x ⃗ itself, but rather an approximation of the expectation value of some operator associated with x ⃗, for example x ⃗^† Mx ⃗ for some matrix M. In the case of sparse matrices where the A matrix is N-by-N, the matrix has a condition number k which determines the time complexity [3]. In the case of classical algorithms, they’re able to achieve a time complexity of O(N√k). The complexity achieved in the paper is a polynomial time of O(log⁡(N,k)) which is an exponential speedup over the classical approach [3].


In [1]:
import math
import scipy
import warnings

import numpy as np

from typing import Optional, Callable, Union, Tuple, List

from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.algorithms import AmplitudeEstimation, EstimationProblem
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.circuit.library import PhaseEstimation

from qiskit.opflow import (
    Z,
    I,
    StateFn,
    TensoredOp,
    ExpectationBase,
    CircuitSampler,
    ListOp,
    ExpectationFactory,
    ComposedOp,
)

from linear_solvers.observables.linear_system_observable import LinearSystemObservable
from linear_solvers import LinearSolver, LinearSolverResult, NumPyLinearSolver
from linear_solvers import HHL as TrueHHL
from linear_solvers.matrices.numpy_matrix import NumPyMatrix

/var/folders/j5/zjbd__c92h76zqc6n0yw0msw0000gn/T/ipykernel_6825/2926853792.py:10: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import AmplitudeEstimation, EstimationProblem


In [2]:
warnings.filterwarnings('ignore', category=DeprecationWarning)

# 1. State Preparation

## random_diag

Generate random diagonal matrix with given properties

Borrowed from qiskit aqua.


- **N** (int): size of matrix
- **eigs** (Union(list, tuple, np.ndarray)): list of N eigenvalues. Overrides K, eigrange.
- **K** (Union(float, list, tuple()): condition number. Either use only condition
    number K or list/tuple of (K, lmin) or (K,
    lmin, sgn). Where lmin is the smallest
    eigenvalue and sign +/- 1 specifies if
    eigenvalues can be negative.
- **eigrange** (Union(list, tuple, nd.ndarray)): [min, max] list for eigenvalue
                                           range. (default=[0, 1])

**Raises** ValueError: invalid input data

**Returns** np.ndarray: diagonal matrix

In [3]:
def random_diag(N, eigs=None, K=None, eigrange=None) -> np.ndarray:  # pylint: disable=invalid-name
    # pylint: disable=invalid-name
    eigrange = eigrange if eigrange is not None else [0, 1]
    if not isinstance(eigs, np.ndarray):
        if eigs is None:
            if not isinstance(K, np.ndarray) and K is not None:
                if isinstance(K, (int, float)):
                    k, lmin, sgn = K, 1, 1
                elif len(K) == 2:
                    k, lmin = K
                    sgn = 1
                elif len(K) == 3:
                    k, lmin, sgn = K
                eigs = aqua_globals.random.random(N)
                a = (k - 1) * lmin / (max(eigs) - min(eigs))
                b = lmin * (max(eigs) - k * min(eigs)) / (max(eigs) - min(eigs))
                eigs = a * eigs + b
                if sgn == -1:
                    sgs = np.random.random(N) - 0.5
                    while min(sgs) > 0 or max(sgs) < 0:
                        sgs = aqua_globals.random.random(N) - 0.5
                    eigs = eigs * (sgs / abs(sgs))
            elif isinstance(eigrange, (tuple, list, np.ndarray)) \
                    and len(eigrange) == 2:
                eigs = \
                    np.random.random(N) * (eigrange[1] - eigrange[0]) + eigrange[0]
            else:
                raise ValueError("Wrong input data: either 'eigs', 'K' or"
                                 "'eigrange' needed to be set correctly.")
        else:
            assert len(eigs) == N, "NxN matrix needs N eigenvalues."
            eigs = np.array(list(eigs))
    else:
        assert len(eigs) == N, "NxN matrix needs N eigenvalues."
    return np.diag(eigs)

In [4]:
m = random_diag(2)

assert m.shape == (2,2)
assert m[1][0] == 0

## random_hermitian

Generate random hermitian (sparse) matrix with given properties. Sparsity is
achieved by truncating Pauli matrices. Sparsity settings alternate the
eigenvalues due to truncation.

Borrowed from qiskit aqua.

- **N** (int): size of matrix
- **eigs** (Union(list, tuple, np.ndarray)): list of N eigenvalues. Overrides K, eigrange
- **K** (Union(float, list, tuple)): condition number. Either use only condition
    number K or list/tuple of (K, lmin) or (K,
    lmin, sgn). Where lmin is the smallest
    eigenvalue and sign +/- 1 specifies if
    eigenvalues can be negative.
- **eigrange** (Union(list, tuple, nd.ndarray)): [min, max] list for eigenvalue
                                           range. (default=[0, 1])
- **trunc** (int): limit for number of Pauli matrices.
- **sparsity** (float): sparsity of matrix. Overrides trunc.

**raises**: ValueError: invalid matrix

**returns** np.ndarray: hermitian matrix

In [5]:
def random_hermitian(N, eigs=None, K=None,  # pylint: disable=invalid-name
                     eigrange=None, sparsity=None,
                     trunc=None) -> np.ndarray:
    
    # pylint: disable=invalid-name
    eigrange = eigrange if eigrange is not None else [0, 1]
    if N == 1:
        raise ValueError('The matrix dimension must be larger than 1')
    u = scipy.stats.unitary_group.rvs(N)
    d = random_diag(N, eigs, K, eigrange)
    ret = u.conj().T.dot(d).dot(u)
    if sparsity or trunc:
        ret = limit_paulis(ret, trunc, sparsity)
    return ret

In [6]:
m = random_hermitian(2)
assert m.shape == (2,2)
assert scipy.linalg.ishermitian(m,atol=5e-11)

## resize_matrix

Resizes the matrix to match the appropriate dimensionality. Fills in the new cells with zeros.

- **matrix** np.ndarray: Square ($N \times N$) or vector (1xN)
- **n** int: The desired dimensionality in $2^n$
- **verbose** bool: Verbosity

**returns** np.ndarray: Resized matrix $2^n \times 2^n$

In [7]:
def resize_matrix(matrix: np.ndarray, n: int, verbose: bool = False) -> np.ndarray:
    dim = 2 ** n

    assert matrix.ndim <= 2, "Currently, only 2-Dimensional arrays or lower are supported."

    if verbose:
        print("Dimensions: ", matrix.ndim)

    # Not square or vector
    # if matrix.ndim >= 2:
    #     assert (matrix.shape[0] == matrix.shape[1] and (matrix.shape[0] == 1 or matrix.shape[1] == 1)), "Matrix is not square or vector"
    # else:

    # Check if shape is (1, 2^n) or (2^n, 2^n)
    # n,m = matrix.shape
    new_shape = tuple(dim for _ in range(matrix.ndim))
    R = np.empty(new_shape)
    X=matrix
    
    if matrix.ndim == 1:
        R[:X.shape[0]] = X[:R.shape[0]]
    elif matrix.ndim == 2:
        R[:X.shape[0],:X.shape[1]] = X[:R.shape[0],:R.shape[1]]
    # matrix = np.resize(matrix, [dim for _ in range(matrix.ndim)])

    return R

    # np.resize(matrix,)

In [8]:
test_vector = np.ones(2)
test_m2 = np.ones((2,2))
test_m3 = np.ones((3,3))

assert np.shape(resize_matrix(test_vector, 1)) == test_vector.shape # Test no changes
assert np.shape(resize_matrix(test_vector, 2)) == (2**2,) # Test 2**2

assert resize_matrix(test_m2, 1).shape == test_m2.shape
assert resize_matrix(test_m2, 2).shape == (4,4)

assert resize_matrix(test_m3, 2).shape == (4,4)

assert resize_matrix(test_m2, 4).shape == (2**4, 2**4)

## calc_qubits

Given a vector, calculates the number of qubits needed to encode it into a circuit.

In [9]:
def calc_qubits(vector: np.ndarray) -> int:
    return math.ceil(np.log2(len(vector)))

In [10]:
assert calc_qubits([0, 0]) == 1
assert calc_qubits([0,0,0]) == 2
assert calc_qubits([0,0,0,0,0]) == 3

## AE_Circuit

Amplitude Encoding Circuit.

- **vector** np.ndarray: Vector $b$.
- **nb** int: number of qubits to represent the vector. Optional, and will be calculated otherwise.
- **verbose** bool: Verbosity

**Returns** QuantumCircuit: the quantum circuit representing the amplitude encoding of the vector.

In [11]:
def AE_Circuit(vector: np.ndarray, nb: int = None, verbose: bool = False) -> QuantumCircuit:
    if nb is None:
        nb = calc_qubits(vector)

        if verbose:
            print(f"Number of Qubits: {nb}")

    if len(vector) != 2**nb:
        vector = resize_matrix(vector, nb, verbose=verbose)

    vector_circuit = QuantumCircuit(nb)

    isometry = vector / np.linalg.norm(vector)

    if verbose:
        print("Isometry: ", isometry)

    vector_circuit.isometry(
        isometry, list(range(nb)), None
    )

    vector_circuit.name = "Amplitude Encoding"
        
    return vector_circuit
    

In [12]:
assert AE_Circuit(np.array([1, 0])).num_qubits == 1
assert AE_Circuit(np.array([1,0.1,1])).num_qubits == 2
assert AE_Circuit(np.array([1,0,1,0.2,0.3])).num_qubits == 3

# 2. Quantum Phase Estimation

## create_matrix_circuit


- **matrix** np.ndarray: Input matrix $A$. Should be a square hermitian
- **number_qubits** int: The expected number of qubits for the circuit

**Returns**: A QuantumCircuit representing the matrix A.

In [13]:
def create_matrix_circuit(matrix: np.ndarray, number_qubits: int) -> QuantumCircuit:
    if isinstance(matrix, list):
        matrix = np.array(matrix)

    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Input matrix must be square!")
    if np.log2(matrix.shape[0]) % 1 != 0:
        raise ValueError("Input matrix dimension must be 2^n!")
    if not np.allclose(matrix, matrix.conj().T):
        raise ValueError("Input matrix must be hermitian!")
    if matrix.shape[0] != 2**number_qubits:
        raise ValueError(
            "Input vector dimension does not match input "
            "matrix dimension! Vector dimension: "
            + str(number_qubits)
            + ". Matrix dimension: "
            + str(matrix.shape[0])
        )
    matrix_circuit = NumPyMatrix(matrix, evolution_time=2 * np.pi)

    return matrix_circuit

In [14]:
for i in [1, 2, 3]:
    n = 2**i
    m = np.round(np.random.random((n,n)) * 10,2)
    m = m + m.conj().T
    num_bits = calc_qubits(m)

    circuit = create_matrix_circuit(m, num_bits)

    assert(circuit.num_qubits == num_bits)

    print(circuit)

# Test non-hermitian
m = np.round(np.random.random((2,2)) * 10,2)
try:
    create_matrix_circuit(m, 2)
except ValueError as e:
    assert str(e) == "Input matrix must be hermitian!"

       ┌─────────────┐
state: ┤ circuit-130 ├
       └─────────────┘
         ┌──────────────┐
state_0: ┤0             ├
         │  circuit-134 │
state_1: ┤1             ├
         └──────────────┘
         ┌──────────────┐
state_0: ┤0             ├
         │              │
state_1: ┤1 circuit-138 ├
         │              │
state_2: ┤2             ├
         └──────────────┘


## calculate_nl

Update the number of qubits required to represent the eigenvalues

$$e^{-2 \pi i \lambda} = e^{2 \pi i (1 - \lambda)}$$

- **number_bits** int: The number of qubits to represent the eigenvalues.
- **kappa** float:
- **neg_vals** float: To register negative eigenvalus. Default true

**Returns** float: The value of the scaling factor.

In [15]:
def calculate_nl(number_bits: int, kappa: float, neg_vals: bool = True):
    nl = max(number_bits + 1, int(np.ceil(np.log2(kappa + 1)))) + neg_vals
    return nl

In [16]:
# Testing

## get_delta

Calculates the scaling factor to represent exactly $\lambda_{min}$ on $n_l$ binary digits.

- **n_l** int: The number of qubits to represent the eigenvalues.
- **lambda_min** float: the smallest eigenvalue.
- **lambda_max** float: the largest eigenvalue.

**Returns** float: The value of the scaling factor.

In [17]:
def get_delta(n_l: int, lambda_min: float, lambda_max: float) -> float:
    formatstr = "#0" + str(n_l + 2) + "b"
    lambda_min_tilde = np.abs(lambda_min * (2**n_l - 1) / lambda_max)
    # floating point precision can cause problems
    if np.abs(lambda_min_tilde - 1) < 1e-7:
        lambda_min_tilde = 1
    binstr = format(int(lambda_min_tilde), formatstr)[2::]
    lamb_min_rep = 0
    
    for i, char in enumerate(binstr):
        lamb_min_rep += int(char) / (2 ** (i + 1))
    return lamb_min_rep

In [18]:
# Testing

## get_kappa

Calculates/retrieves the kappa value for the circuit.

- **matrix_circuit** QuantumCircuit: The quantum circuit representing $A$.

**Returns** float: the kappa value

In [19]:
def get_kappa(matrix_circuit: QuantumCircuit) -> float:
    if (
        hasattr(matrix_circuit, "condition_bounds")
        and matrix_circuit.condition_bounds() is not None
    ):
        kappa = matrix_circuit.condition_bounds()[1]
    else:
        kappa = 1

    return kappa

In [20]:
# Testing

## set_eigenbounds

Calculates and sets the eigenbounds for the encoded matrix circuit.

- **matrix_circuit** QuantumCircuit: The quantum circuit representing $A$.
- **nl** int: Adjusted number of qubits for the A circuit.
- **neg_vals** bool: Whether to allow negative values.

**Returns** Tuple[QuantumCircuit, float, float]: Returns the updated matrix circuit, the delta, and the scaling values

In [21]:
def set_eigenbounds(matrix_circuit: QuantumCircuit, nl: int, neg_vals: bool = True) -> Tuple[QuantumCircuit, float, float]:
    if (hasattr(matrix_circuit, "eigs_bounds") and matrix_circuit.eigs_bounds() is not None):
        
        lambda_min, lambda_max = matrix_circuit.eigs_bounds()
        # Constant so that the minimum eigenvalue is represented exactly, since it contributes
        # the most to the solution of the system. -1 to take into account the sign qubit
        delta = get_delta(nl - neg_vals, lambda_min, lambda_max)
        # Update evolution time
        matrix_circuit.evolution_time = (
            2 * np.pi * delta / lambda_min / (2**neg_vals)
        )
        # Update the scaling of the solution
        scaling = lambda_min
    else:
        delta = 1 / (2**nl)
        scaling = 1
        print("The solution will be calculated up to a scaling factor.")

    return matrix_circuit, delta, scaling


In [22]:
# Testing

## calculate_norm

Calculates the value of the euclidean norm of the solution.

- **qc** QuantumCircuit: The quantum circuit preparing the solution x to the system.
- **scaling** float: Scaling factor of the norm

**Returns** float: The value of the euclidean norm of the solution.

In [23]:
def calculate_norm(qc: QuantumCircuit, scaling: float) -> float:
    # Calculate the number of qubits
    nb = qc.qregs[0].size
    nl = qc.qregs[1].size
    na = qc.num_ancillas

    # Create the Operators Zero and One
    zero_op = (I + Z) / 2
    one_op = (I - Z) / 2

    # Norm observable
    observable = one_op ^ TensoredOp((nl + na) * [zero_op]) ^ (I ^ nb)
    norm_2 = (~StateFn(observable) @ StateFn(qc)).eval()

    return np.real(np.sqrt(norm_2) / scaling)

In [24]:
# Testing

# 3. Production

## construct_circuit

Constructs the HHL circuit using 
- <a href="#calc_qubits">calc_qubits</a>: Calculating the number of qubits
- <a href="#create_matrix_circuit">create_matrix_circuit</a>: Creating and encoding the matrix circuit
- <a href="#get_kappa">get_kappa</a>: Calculates Kappa
- <a href="#create_matrix_circuit">create_matrix_circuit</a>: Encodes $A$ into a circuit
- <a href="#set_eigenbounds">set_eigenbounds</a>: Recalculates eigenbounds
- <a href="#calculate_nl">calculate_nl</a>: Recalculates $n_l$.

Using these supporting functions, construct_circuit encodes the data from the inputs and creates the QPE as well as rotaional circuits.

- **A** np.ndarray: Input matrix $A$. Should be a square hermitian
- **b** np.ndarray: Input vector $b$.
- **neg_vals** bool: Whether to allow negative eigenvalues.
- **nf** int: Number of flag qubits.
- **epsilon** flaot: Error threshold

**Returns** QuantumCircuit: The HHL Circuit

In [25]:
def construct_circuit(A: np.ndarray, b: np.ndarray, neg_vals: bool = True, nf: int = 1, epsilon: float = 1e-2) -> QuantumCircuit:
    _epsilon_r = epsilon / 3  # conditioned rotation
    _epsilon_s = epsilon / 3  # state preparation
    _epsilon_a = epsilon / 6  # hamiltonian simulation

    nb = calc_qubits(b)
    
    ae = AE_Circuit(b, nb, verbose=True)
    print(ae)

    matrix_circuit = create_matrix_circuit(A, ae.num_qubits) #???

    kappa = get_kappa(matrix_circuit)
    
    nl = calculate_nl(nb, kappa)

    if hasattr(matrix_circuit, "tolerance"):
        matrix_circuit.tolerance = _epsilon_a

    matrix_circuit, delta, scaling = set_eigenbounds(matrix_circuit, nl)

    phase_estimation = PhaseEstimation(nl, matrix_circuit)

    reciprocal_circuit = ExactReciprocal(nl, delta, neg_vals=neg_vals)
    reciprocal_circuit.name = "Conditional Rotation"
    # Update number of ancilla qubits
    na = matrix_circuit.num_ancillas

    # check if the matrix can calculate bounds for the eigenvalues

    # Initialise the quantum registers
    qb = QuantumRegister(nb)  # right hand side and solution
    ql = QuantumRegister(nl)  # eigenvalue evaluation qubits
    if na > 0:
        qa = AncillaRegister(na)  # ancilla qubits
    qf = QuantumRegister(nf)  # flag qubits

    if na > 0:
        qc = QuantumCircuit(qb, ql, qa, qf)
    else:
        qc = QuantumCircuit(qb, ql, qf)
    
    # State preparation
    qc.append(ae, qb[:])
    
    # QPE
    if na > 0:
        qc.append(phase_estimation, ql[:] + qb[:] + qa[: matrix_circuit.num_ancillas])
    else:
        qc.append(phase_estimation, ql[:] + qb[:])

    # Coditional Rotation
    qc.append(reciprocal_circuit, ql[::-1] + [qf[0]])

    # QPE inverse
    if na > 0:
        qc.append(
            phase_estimation.inverse(),
            ql[:] + qb[:] + qa[: matrix_circuit.num_ancillas],
        )
    else:
        qc.append(phase_estimation.inverse(), ql[:] + qb[:])
    return qc, scaling

In [26]:
# Testing

## HHL

Creates the HHL circuit (<a href="#construct_circuit">construct_circuit</a>), calculates the norm (<a href="#calculate_norm">calculate_norm</a>), and returns a linear solver result.

- **A** np.ndarray: Input matrix $A$. Should be a square hermitian
- **b** np.ndarray: Input vector $b$.
- **neg_vals** bool: Whether to allow negative eigenvalues.
- **nf** int: Number of flag qubits.
- **epsilon** flaot: Error threshold

**Returns** LinearSolverResult: The HHL Solution

In [27]:
def HHL(A: np.ndarray, b: np.ndarray, neg_vals: bool = True, nf: int = 1, epsilon: float = 1e-2) -> LinearSolverResult:
    qc, scaling = construct_circuit(A, b, neg_vals, nf, epsilon)

    norm = calculate_norm(qc, scaling)

    sol = LinearSolverResult()
    
    sol.state = qc
    sol.euclidean_norm = norm

    return sol

In [28]:
A = np.array([
    [1, -1/3],
    [-1/3, 1]
])

b = np.array([1, 0])

assert HHL(A,b)
sol = HHL(A, b)
assert round(sol.euclidean_norm,2) == 1.19

Isometry:  [1. 0.]
   ┌──────────┐
q: ┤ Isometry ├
   └──────────┘
Isometry:  [1. 0.]
   ┌──────────┐
q: ┤ Isometry ├
   └──────────┘


## compute_x

Computes the $x$ vector given the stored linear solver result.

- **naive_hhl_solution** LinearSolverResult: Linear Solver result which contains the circuit and data.
- **verbose** bool: Verbosity

**Returns** np.ndarray: The $x$ vector

In [29]:
def compute_x(naive_hhl_solution: LinearSolverResult, verbose: bool = False) -> np.ndarray:

    naive_sv = Statevector(naive_hhl_solution.state).data
    
    # Extract vector components; 10000(bin) == 16 & 10001(bin) == 17
    naive_full_vector = np.array([naive_sv[16], naive_sv[17] ])

    if verbose:
        print('naive raw solution vector:', naive_full_vector)

    def get_solution_vector(solution):
        """Extracts and normalizes simulated state vector
        from LinearSolverResult."""
        solution_vector = Statevector(solution.state).data[16:18].real
        norm = solution.euclidean_norm
        return norm * solution_vector / np.linalg.norm(solution_vector)

    solution = get_solution_vector(naive_hhl_solution)
    if verbose:
        print('full naive solution vector:', solution)

    return solution

In [30]:
# Testing

## hhl_solver

Creates the HHL circuit (<a href="#construct_circuit">construct_circuit</a>), calculates the norm (<a href="#calculate_norm">calculate_norm</a>), and returns a linear solver result.

- **A** np.ndarray: Input matrix $A$. Should be a square hermitian
- **b** np.ndarray: Input vector $b$.
- **neg_vals** bool: Whether to allow negative eigenvalues.
- **nf** int: Number of flag qubits.
- **epsilon** flaot: Error threshold

**Returns** np.ndarray: The $x$ vector

In [31]:
def hhl_solver(A: np.ndarray, b: np.ndarray, neg_vals: bool = True, nf: int = 1, epsilon: float = 1e-2) -> np.ndarray:

    solution = HHL(
        A=A, 
        b=b,
        neg_vals=neg_vals,
        nf=nf,
        epsilon=epsilon
    )

    return compute_x(solution)

In [32]:
# Testing

# 4. Testing

Now that the functions for the HHL algorithm are defined, I can run it on a number of test cases

In [33]:
a_tests = [
    np.array([ [1, -1/3], [-1/3, 1] ])
]

b_tests = [
    np.array([1, 0])
]

x_tests = [
    np.array([1.125, 0.375])
]

for n in 2**np.array([1]):
    a = random_hermitian(n).real
    b = np.round(np.random.random(n) * 10,2)

    print(a)

    # Norm b
    b = b / np.linalg.norm(b)

    x = np.linalg.solve(a,b)

    assert np.allclose(np.dot(a, x), b)

    a_tests.append(a)
    b_tests.append(b)
    x_tests.append(x)
print(x_tests)

[[ 0.57450281 -0.08154984]
 [-0.08154984  0.83999027]]
[array([1.125, 0.375]), array([1.7688714 , 0.36976202])]


In [34]:
for a, b, x in zip(a_tests, b_tests, x_tests):
    naive_hhl_solution = TrueHHL().solve(a, b)
    print("LinSolver Answer: ", compute_x(naive_hhl_solution))
    answer = hhl_solver(a, b)

    print("X: ", x)

    print("Answer: ", answer)
    print(np.allclose(np.dot(a, answer), b))
    print("-----")

LinSolver Answer:  [1.125 0.375]
Isometry:  [1. 0.]
   ┌──────────┐
q: ┤ Isometry ├
   └──────────┘
X:  [1.125 0.375]
Answer:  [1.125 0.375]
True
-----
LinSolver Answer:  [1.76424364 0.38613576]
Isometry:  [0.98606756 0.16634531]
   ┌──────────┐
q: ┤ Isometry ├
   └──────────┘
X:  [1.7688714  0.36976202]
Answer:  [1.76424364 0.38613576]
False
-----


## Try for Yourself

The common example

In [35]:
A = np.array([ [1, -1/3], [-1/3, 1] ])

b = np.array([1, 0])

x = np.array([1.125, 0.375])

solution = HHL(A, b)

calculated_x = compute_x(solution)
print("calculated x = ", calculated_x)
print("True x = ", x)

solution.state.draw()

Isometry:  [1. 0.]
   ┌──────────┐
q: ┤ Isometry ├
   └──────────┘
calculated x =  [1.125 0.375]
True x =  [1.125 0.375]


┌────────────────────┐┌──────┐                         ┌─────────┐
  q185: ┤ Amplitude Encoding ├┤3     ├─────────────────────────┤3        ├
        └────────────────────┘│      │┌───────────────────────┐│         │
q186_0: ──────────────────────┤0     ├┤2                      ├┤0        ├
                              │  QPE ││                       ││  QPE_dg │
q186_1: ──────────────────────┤1     ├┤1                      ├┤1        ├
                              │      ││  Conditional Rotation ││         │
q186_2: ──────────────────────┤2     ├┤0                      ├┤2        ├
                              └──────┘│                       │└─────────┘
  q187: ──────────────────────────────┤3                      ├───────────
                                      └───────────────────────┘

A random matrix that works okay, but not well.

In [36]:
A = np.array([[ 0.68559428, -0.11680403],
       [-0.11680403,  0.86405785]])

b = np.array([0.99399629, 0.10941377])

x = np.array([1.50609134, 0.33022247])

solution = HHL(A, b)

calculated_x = compute_x(solution)
print("calculated x = ", calculated_x)
print("True x = ", x)

solution.state.draw()

Isometry:  [0.99399629 0.10941377]
   ┌──────────┐
q: ┤ Isometry ├
   └──────────┘
calculated x =  [1.5012976  0.33991712]
True x =  [1.50609134 0.33022247]


┌────────────────────┐┌──────┐                         ┌─────────┐
  q215: ┤ Amplitude Encoding ├┤3     ├─────────────────────────┤3        ├
        └────────────────────┘│      │┌───────────────────────┐│         │
q216_0: ──────────────────────┤0     ├┤2                      ├┤0        ├
                              │  QPE ││                       ││  QPE_dg │
q216_1: ──────────────────────┤1     ├┤1                      ├┤1        ├
                              │      ││  Conditional Rotation ││         │
q216_2: ──────────────────────┤2     ├┤0                      ├┤2        ├
                              └──────┘│                       │└─────────┘
  q217: ──────────────────────────────┤3                      ├───────────
                                      └───────────────────────┘